In [13]:
%load_ext autoreload
%autoreload 2

# load packages
import numpy as np
import time

# load local functions
import tools

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Exercise 5: Numerical Integration
Consider the integration problem, arising from computing the expectation of $f(X)=X^2$, where $X \sim \mathcal{N}(0,1)$:
$$
\mathbb{E}[X^2] = \int x^{2}p(x)dx
$$
For this particular integral, we have an analytical solution, which follows from the variance formula:
$$1=Var(X)=\mathbb{E}[X^2]- {\underbrace{\mathbb{E}[X]}_{=0}}^2 \Rightarrow \mathbb{E}[X^2]=1$$
, which makes it suitable to compare approximations with.

In [6]:
# Define the function 
f = lambda x: x**2

### 1. Approximate the integral using *Monte Carlo integration*.

In [7]:
num_draws = 1000 # number of MC draws
np.random.seed(2026) # set seed to make sure the results are the same
x_mc = np.random.normal(size=num_draws) # draw from standard normal distribution
Efx_MC = np.mean(f(x_mc))
print(Efx_MC)

1.0593351204408865


In [8]:
#Create a function for later use
def integrate_MC(f,num_points):
    np.random.seed(2026)
    x = np.random.normal(size=num_points) 
    return np.mean(f(x))

### 2. Approximate the integral using *Gauss-Hermite integration*.

Gauss-Hermite approximation:

\begin{align*}
    \int_{-\infty}^{\infty} f(x) \exp\{-x^2\} dx = \sum^n_{i=1} \omega_{i} f(x_{i}) +\text{error term}
\end{align*}

Hint: The goal is to rewrite the integration problem $\int x^{2}p(x) dx=\int x^2 \frac{1}{\sqrt{2 \pi}} \exp \left( - \frac{x^2}{2} \right) dx$, with a change of variable such that $g(z) = \exp\{-x^2\}$, where z is the change of variable that makes the approximation work.

Solution:

Use $z=\frac{x}{\sqrt{2}} \Leftrightarrow x = \sqrt{2}z $ and $dz=\frac{1}{\sqrt{2}}dx \Leftrightarrow dx = \sqrt{2} dz $ such that:
$$\begin{align*}
\int{ x^2 \frac{1}{\sqrt{2 \pi}} \exp{ \left( -\frac{x^2}{2} \right) } dx } &= \int{ 2z^2 \frac{1}{\sqrt{2 \pi}} \exp{ \left( -z^2 \right) }\sqrt{2} dz} \\
&= \int{ (\sqrt{2}z)^2 \frac{1}{\sqrt{\pi}} \exp{ \left( -z^2 \right) } dz}
\end{align*}$$

, i.e. multiply $z$ ("raw" from Gauss-Hermite) by $\sqrt{2}$ and divide weights ("raw" from Gauss-Hermite) by $\sqrt{\pi}$ and evaluate the sum to approximate the integral

For a general derivation when $X \sim \mathcal{N}(\mu, \sigma^2)$ and intuition, see the slides for the exercise classes.

In [10]:
num_points = 5

# get "raw" hermite nodes and weights
x_gauss,w_gauss = tools.gauss_hermite(num_points)

# adjust accordingly to the distribution X is drawn from. Here, standard Gaussian
# FILL IN. Hint: Use derivation above / exercise class slides

### SOLUTION ###
x_gauss = x_gauss*np.sqrt(2)
w_gauss = w_gauss/np.sqrt(np.pi)
### SOLUTION ###

# evaluate expectation
Efx_gauss = f(x_gauss.T) @ w_gauss
print(Efx_gauss)

1.000000000000002


In [11]:
# construct function for use below
def integrate_gauss(f,num_points):
    x_gauss,w_gauss = tools.gauss_hermite(num_points)
    # FILL IN

    ### SOLUTION ###
    x_gauss = x_gauss*np.sqrt(2)
    w_gauss = w_gauss/np.sqrt(np.pi)
    ### SOLUTION ###
    
    return f(x_gauss.T) @ w_gauss

### 3. Compare the two methods across various number of grid points. How few grid points do you need for Gauss-Hermite integration?

In [14]:
num_array = [1,2,3,10,50,100,1000,3000,900000]

for i,num in enumerate(num_array):

    # set the starting time
    t0 = time.time()  
    print(f'Number of grid points:    {num}')

    # do monte carlo
    print(f'MC:    {integrate_MC(f,num):.4f}')

    # do gauss-hermite if there aren't too many points
    if num < 1500:
        print(f'gauss: {integrate_gauss(f,num):.4f}')
    
    # print true value
    print(f'True value:  {1:.4f}')

    # print the total time
    print(f'time: {time.time()-t0:.8} seconds\n') 

Number of grid points:    1
MC:    0.1864
gauss: 0.0000
True value:  1.0000
time: 0.00039505959 seconds

Number of grid points:    2
MC:    1.0632
gauss: 1.0000
True value:  1.0000
time: 0.00015926361 seconds

Number of grid points:    3
MC:    0.7412
gauss: 1.0000
True value:  1.0000
time: 0.00040388107 seconds

Number of grid points:    10
MC:    0.8128
gauss: 1.0000
True value:  1.0000
time: 0.00015306473 seconds

Number of grid points:    50
MC:    1.0877
gauss: 1.0000
True value:  1.0000
time: 0.00064516068 seconds

Number of grid points:    100
MC:    1.0236
gauss: 1.0000
True value:  1.0000
time: 0.0077571869 seconds

Number of grid points:    1000
MC:    1.0593
gauss: 1.0000
True value:  1.0000
time: 1.0827689 seconds

Number of grid points:    3000
MC:    1.0524
True value:  1.0000
time: 0.0002989769 seconds

Number of grid points:    900000
MC:    0.9991
True value:  1.0000
time: 0.018042803 seconds



### 4. Change the function f and see what happens.

In [15]:
num_array = [1,2,3,4,5,10,50,100,500,3000]

# New function
g = lambda x: np.exp(x)

for i,num in enumerate(num_array):
    print(f'Number of grid points:    {num}')
    if num < 1500:
        print(f'gauss: {integrate_gauss(g,num):.4f}')
    print(f'MC:    {integrate_MC(g,num):.4f}')
    print(f'True value:  {np.exp(1/2):.4f}')
    print(f'')

Number of grid points:    1
gauss: 1.0000
MC:    0.6494
True value:  1.6487

Number of grid points:    2
gauss: 1.5431
MC:    0.4489
True value:  1.6487

Number of grid points:    3
gauss: 1.6382
MC:    0.7544
True value:  1.6487

Number of grid points:    4
gauss: 1.6480
MC:    0.8125
True value:  1.6487

Number of grid points:    5
gauss: 1.6487
MC:    1.5024
True value:  1.6487

Number of grid points:    10
gauss: 1.6487
MC:    1.2073
True value:  1.6487

Number of grid points:    50
gauss: 1.6487
MC:    1.7514
True value:  1.6487

Number of grid points:    100
gauss: 1.6487
MC:    1.6859
True value:  1.6487

Number of grid points:    500
gauss: 1.6487
MC:    1.7079
True value:  1.6487

Number of grid points:    3000
MC:    1.7633
True value:  1.6487

